#**Data anlaytics fundamentals assignment**
Data is loaded from:

"Anonymised voids project data.xlsx"

Random Forest regressor will applied on the data set.

## Data loading


Load the data from the provided Excel file into a pandas DataFrame.


###Code

In [11]:
import pandas as pd

try:
    df = pd.read_excel("Anonymised voids project data.xlsx")
    display(df.head())
    print(df.shape)
except FileNotFoundError:
    print("Error: 'Anonymised voids project data.xlsx' not found.")
    df = None
except Exception as e:
    print(f"An error occurred: {e}")
    df = None

,Tenancy_ID,Property_ID,Property Type,Tenancy Commencement Date,Tenancy Termination Date,Construction Year,Prev_Ten_ID,#_of_Responsive_Repairs,#_of_Block_Repairs,#_of_planned_repairs,#_of_other_repairs,#_of_Complaint_Cases_against_Prev_Ten,IsVoid?
0,23662,13748,3 Bed 3rd flr lifted Maisonett,2010-09-06,NaT,1958,23662,0,0,0,0,3,False
1,23664,12295,1 Bed 21st floor lifted flat,2010-09-06,NaT,1997,23664,0,0,0,0,0,False
2,23674,11251,2 Bed 4th floor lifted Mais,2010-09-13,NaT,1957,23674,4,0,0,0,0,False
3,23681,10909,1 Bed 1st flr unlifted flat,2010-09-13,NaT,1960,23681,1,0,0,0,0,False
4,23691,11271,2 Bed 1st floor lifted flat,2010-09-20,NaT,1948,23691,0,0,0,0,1,False


(1957, 13)


## Data exploration


Explore the loaded data to understand its characteristics.


###Code

In [12]:
# Data Types
print("Data Types:\n", df.dtypes)

# Missing Values
missing_values = df.isnull().sum()
missing_percentage = (missing_values / len(df)) * 100
print("\nMissing Values:\n", missing_values)
print("\nMissing Value Percentage:\n", missing_percentage)

# Summary Statistics for Numerical Columns
numerical_cols = df.select_dtypes(include=['number'])
print("\nSummary Statistics for Numerical Columns:\n", numerical_cols.describe())

# Distribution of Categorical Columns
categorical_cols = df.select_dtypes(include=['object', 'bool'])
for col in categorical_cols.columns:
    print(f"\nDistribution of {col}:\n", df[col].value_counts())

# Data Shape
print(f"\nData Shape: {df.shape}")

# Check for inconsistencies (example: tenancy termination date after commencement date)
# Convert date columns to datetime objects if they aren't already
for col in ['Tenancy Commencement Date', 'Tenancy Termination Date']:
    df[col] = pd.to_datetime(df[col], errors='coerce')

inconsistent_dates = df[df['Tenancy Termination Date'] < df['Tenancy Commencement Date']]
print("\nInconsistencies (Termination Date before Commencement Date):\n", inconsistent_dates)

Data Types:
 Tenancy_ID                                        int64
Property_ID                                       int64
Property Type                                    object
Tenancy Commencement Date                datetime64[ns]
Tenancy Termination Date                 datetime64[ns]
Construction Year                                 int64
Prev_Ten_ID                                       int64
#_of_Responsive_Repairs                           int64
#_of_Block_Repairs                                int64
#_of_planned_repairs                              int64
#_of_other_repairs                                int64
#_of_Complaint_Cases_against_Prev_Ten             int64
IsVoid?                                            bool
dtype: object

Missing Values:
 Tenancy_ID                                  0
Property_ID                                 0
Property Type                               0
Tenancy Commencement Date                   0
Tenancy Termination Date                 10

###**Main Takeaways**



More than 50% of tenancy termination date is missing. This is expected as all non voids will not have a termination date

Some Properties do not have a construction year. These rows will purged. The percentage of rows without a construction year is low so it will not affect the sixe of the data set.

Property type attribute needs to change datatype from string

## Data cleaning

Clean the data by handling missing values and inconsistencies.


In [13]:

import numpy as np

# Impute missing 'Tenancy Termination Date' with a far future date
future_date = pd.to_datetime('2099-12-31')
df['Tenancy Termination Date'] = df['Tenancy Termination Date'].fillna(future_date)

# Address inconsistencies in 'Construction Year'
# Remove rows with custruction year = 0
df = df[df['Construction Year'] != 0]

# Remove duplicate rows
df.drop_duplicates(inplace=True)

# Convert data types (if necessary)
# Example: Convert 'Property Type' to category
df['Property Type'] = df['Property Type'].astype('category')

#Replacing header names to not include spaces
df.columns = df.columns.str.replace(' ', '_')

# Check for missing values and data types again after cleaning
print("\nMissing Values:\n", df.isnull().sum())
print("\nData Types:\n", df.dtypes)


Missing Values:
 Tenancy_ID                               0
Property_ID                              0
Property_Type                            0
Tenancy_Commencement_Date                0
Tenancy_Termination_Date                 0
Construction_Year                        0
Prev_Ten_ID                              0
#_of_Responsive_Repairs                  0
#_of_Block_Repairs                       0
#_of_planned_repairs                     0
#_of_other_repairs                       0
#_of_Complaint_Cases_against_Prev_Ten    0
IsVoid?                                  0
dtype: int64

Data Types:
 Tenancy_ID                                        int64
Property_ID                                       int64
Property_Type                                  category
Tenancy_Commencement_Date                datetime64[ns]
Tenancy_Termination_Date                 datetime64[ns]
Construction_Year                                 int64
Prev_Ten_ID                                       int64
#_of

<ipython-input-13-3f49273d79e9>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(inplace=True)
<ipython-input-13-3f49273d79e9>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Property Type'] = df['Property Type'].astype('category')


##Applying Random forest regression model

###Imports and setting static variables

In [14]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier
from sklearn.metrics import mean_squared_error
from sklearn.metrics import confusion_matrix

target_attribute = 'IsVoid?'
predictor_attributes = ['Property_Type', 'Construction_Year', '#_of_Responsive_Repairs',
                       '#_of_Block_Repairs', '#_of_planned_repairs', '#_of_other_repairs',
                       '#_of_Complaint_Cases_against_Prev_Ten']

X = df[predictor_attributes]
y = df[target_attribute]

X = pd.get_dummies(X, columns=['Property_Type'], drop_first=True)


###Model parameters
These parameters will be subject to change to improve the accuracy of the model

In [15]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create the Random Forest Regression model
model = RandomForestRegressor(random_state=42)

###Fitting and evaluating model

In [16]:
# Train the Random Forest Regression model
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")


Mean Squared Error: 0.10454977359347728


###Findings
It seems with a mean squared error of 0.104 the model is very accurate in predicting the liklihood of a property being void or not.

I wonder if it would be the same with a random forest classifier

In [17]:
model2 = RandomForestClassifier(random_state=42)
model2.fit(X_train, y_train)

# Make predictions on the test set
y2_pred = model2.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y2_pred)
print(f"Mean Squared Error: {mse}")
cm1 = confusion_matrix(y_test, y2_pred)
print("Confusion Matrix:\n", cm1)

Mean Squared Error: 0.13402061855670103
Confusion Matrix:
 [[178  26]
 [ 26 158]]


In [24]:

import pandas as pd
from sklearn.metrics import confusion_matrix


# Create a DataFrame with the input features

tenancy_ids_test = df.loc[X_test.index, 'Tenancy_ID']
property_ids_test = df.loc[X_test.index, 'Property_ID']

output_df = pd.DataFrame(X_test)
output_df['Tenancy_ID'] = tenancy_ids_test
output_df['Property_ID'] = property_ids_test


# Add the actual target variable
output_df['Actual_IsVoid'] = y_test

# Add the predicted target variable
output_df['Predicted_IsVoid'] = y_pred



# Save the DataFrame to a CSV file
output_df.to_csv('test_set_predictions.csv', index=False)
output_df['Tenancy_ID']

,Tenancy_ID
588,28518
912,30187
918,30230
1775,29473
1482,27862
...,...
1933,30508
1658,28982
1275,26025
534,28102
